In [1]:
import os
print(os.getcwd())  # this is where Jupyter is looking from
#one folder back
os.chdir('..')
print(os.getcwd())

c:\Users\Christian\Documents\Cand_merc\n1_masters_thesis\model_training
c:\Users\Christian\Documents\Cand_merc\n1_masters_thesis


In [13]:
import pandas as pd
import numpy as np
from load_data import load_dataset, ImageDataset

from split import get_dataloaders
from custom_data_aug import get_transforms

import torch
import torch.nn as nn
import torchvision
from config import CFG

from torchinfo import summary

from torch.utils.tensorboard import SummaryWriter
from training import train  # or wherever your updated function lives

from acc_loss_plot import plot_training_curves

In [3]:
# Load the dataframe
csv_path = "C:/Users/Christian/Desktop/N1_data/filtered_df.csv"
image_dir = "C:/Users/Christian/Desktop/N1_data/image_data/"
df, idx_to_label = load_dataset(csv_path, image_dir)

In [4]:
print(df.head())

                                           file_name label  label_idx
0  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
1  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
2  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
3  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
4  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0


In [4]:
train_transforms, val_transforms = get_transforms()

train_dataset, val_dataset, train_loader, val_loader = get_dataloaders(
    df, train_transforms, val_transforms
)

In [5]:
def build_model(device: torch.device, num_classes: int = CFG.NUM_CLASSES) -> nn.Module:
    # Set seeds
    torch.manual_seed(CFG.SEED)
    torch.cuda.manual_seed(CFG.SEED)

    weights = torchvision.models.ResNet50_Weights.DEFAULT
    model = torchvision.models.resnet50(weights=weights).to(device)

    for param in model.parameters():
        param.requires_grad = False

    for param in model.layer4.parameters():
        param.requires_grad = True
    
    model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
    model.fc = nn.Sequential(nn.Flatten(),
        nn.Linear(2048, 512),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(512, num_classes)
    ).to(device)

    return model

In [6]:
cnn = build_model(device=CFG.DEVICE)

# View model summary
summary(
    model=cnn, 
    input_size=(CFG.BATCH_SIZE, CFG.CHANNELS, CFG.WIDTH, CFG.HEIGHT),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [64, 3, 224, 224]    [64, 34]             --                   Partial
├─Conv2d (conv1)                         [64, 3, 224, 224]    [64, 64, 112, 112]   (9,408)              False
├─BatchNorm2d (bn1)                      [64, 64, 112, 112]   [64, 64, 112, 112]   (128)                False
├─ReLU (relu)                            [64, 64, 112, 112]   [64, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [64, 64, 112, 112]   [64, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [64, 64, 56, 56]     [64, 256, 56, 56]    --                   False
│    └─Bottleneck (0)                    [64, 64, 56, 56]     [64, 256, 56, 56]    --                   False
│    │    └─Conv2d (conv1)               [64, 64, 56, 56]     [64, 64, 56, 56]     (4,096)              False
│    │    

In [7]:
# Define Loss Function
loss_fn = nn.CrossEntropyLoss(
    label_smoothing=0.1
)

# Define Optimizer
optimizer = torch.optim.Adam(
    cnn.parameters(),
    lr=CFG.LR
)

In [8]:
writer = SummaryWriter(log_dir="runs/exp1_efficientnet")

session = train(
    model=cnn,
    train_dataloader=train_loader,
    eval_dataloader=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=CFG.EPOCHS,
    device=CFG.DEVICE,
    writer=writer
)

writer.close()


Epoch 1/3


  0%|          | 0/117 [00:00<?, ?it/s]

loss: 2.4578 - top1: 0.3168 - top5: 0.7202 - eval_loss: 1.7321 - eval_top1: 0.5445 - eval_top5: 0.9506

Epoch 2/3


  0%|          | 0/117 [00:00<?, ?it/s]

loss: 1.7955 - top1: 0.5240 - top5: 0.9253 - eval_loss: 1.5766 - eval_top1: 0.5878 - eval_top5: 0.9688

Epoch 3/3


  0%|          | 0/117 [00:00<?, ?it/s]

loss: 1.6457 - top1: 0.5845 - top5: 0.9492 - eval_loss: 1.5316 - eval_top1: 0.6538 - eval_top5: 0.9688


In [10]:
# Convert history dict to DataFrame
session_history_df = pd.DataFrame(session)
session_history_df

,loss,accuracy,top5_accuracy,eval_loss,eval_accuracy,eval_top5_accuracy
0,2.457811,0.316850,0.720200,1.732096,0.544471,0.950641
1,1.795486,0.523956,0.925335,1.576558,0.587821,0.968750
2,1.645655,0.584478,0.949233,1.531643,0.653766,0.968750


In [14]:
from acc_loss_plot import plot_training_curves
# Plot EfficientNet session training history 
plot_training_curves(session)

KeyError: 'eval_accuaracy'